In [80]:
import pandas as pd
import pymssql
from clickhouse_driver import Client
from sqlalchemy import create_engine
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [81]:
pd. set_option('display.max_rows', 200)

In [ ]:
def execute_clickhouse(query="select 'Test' as Test",
                       database='mart_',
                       external_tables=None,
                       **kwargs) -> pd.DataFrame:
    """
    Функция для выполнения запросов к Clickhouse

    **Аргументы:**

    - `query: str` - запрос
    - `database: str` - база данных (по-умолчанию `'dns_log'`)
    - `external_tables: pandas.DataFrame` - датафрэйм из которого бдует генерироваться внешняя таблица

    **Возвращает:**
    >    `pandas.DataFrame` с результатом запроса
    """
    client = Client(
        host='',
        port='',
        database=database,
        user='',
        password=''
    )
    if external_tables:
        pass
        #external_tables = [df_to_table(table[0], table[1]) for table in external_tables]
    rv = client.execute(query, external_tables=external_tables, with_column_types=True)
    df = pd.DataFrame(rv[0])
    df.columns = [i[0] for i in rv[1]]
    return df

def execute_pg_hr(query):
    """Функция обращения к хранилищу mart_ DNS на чтение
    Параметры:
    query - строка, SQL запрос
    Возвращает - pandas DataFrame с содержанием результата выполнения запроса, названия столбцов из запроса"""

    host = ''
    schema = ''
    login = ''
    password = ''

    conection_url = f'postgresql+psycopg2://{login}:{password}@{host}/{schema}'

    engine = create_engine(conection_url)
    df = pd.read_sql(query, engine)
    engine.dispose()
    return df

def get_postgres_connection_string(connection_id) -> str:
    """
    Возвращает строку подключения к postgres для указанного подключения
    :return:
    """
    class connection:
        login = ''
        password = ''
        host = ''
        port = ''
        schema = ''

    return f'postgresql+psycopg2://{connection.login}:{connection.password}@{connection.host}:{connection.port}/{connection.schema}'

def execute_onec(query):
    """Функция обращения к Хранилищу DNS на чтение
    Параметры:
    query - строка, SQL запрос
    Возвращает - pandas DataFrame с содержанием результата выполнения запроса, названия столбцов из запроса"""
    ADuser = ''
    ADpassword = ''
    # ADuser = Config.ONEC_LOGIN
    # ADpassword = Config.ONEC_PASSWORD
    conn = pymssql.connect(server='', user=ADuser, password=ADpassword, database='')
    df = pd.read_sql(query, conn)
    
    return df


In [83]:
def GetPosLvlSalary():
    query = f"""                                                                                                                                                                               
	  WITH first_query AS
        (
SELECT  
--		reg_table.Должность AS Должность
        ШтатноеРасписание.Наименование AS Должность
        ,рсузп.Уровень AS Уровень
        ,сг.Наименование AS Город
        ,сг.Код AS КодГорода
        , сг.Регион AS СсылкаРегион
        ,рсузп.Значение AS Зарплата
        , MAX(Период) OVER (PARTITION BY рсузп.Город, рсузп.Должность, рсузп.Уровень) AS max_period
        ,рсузп.Период as Период
        ,ШтатноеРасписание.ЛидерскийУровень
        , рсузп.Город  AS СсылкаГород
        , сг.ТерриториальноеРасположение AS СсылкаГородТерритория
FROM dwh.[РегистрСведений.УровеньЗаработнойПлаты] AS рсузп
JOIN dwh.[Справочник.Города] AS сг 
	ON рсузп.Город = сг.Ссылка 
JOIN dwh.[Справочник.ШтатноеРасписание] ШтатноеРасписание ON рсузп.Должность  =  ШтатноеРасписание.Ссылка  
    AND ШтатноеРасписание.ПрефиксКДолжности NOT IN  ('ДОМ', 'ГРП')      		
        ),
findMaxDate AS (
SELECT 
	  Город
	, КодГорода
	, Должность
	, Уровень
	, Зарплата
	, Период
	, max_period
	, CONVERT(DATE, GETDATE()) AS ДатаОбновления
	, ЛидерскийУровень
	, СсылкаГород
	, СсылкаРегион
	
FROM  first_query
WHERE Период = max_period
    AND Должность != '2.03. Фед.развитие сайта'
    AND Должность not like '%(на аутсорсе)%' 
        ),
joinTerrerory AS (
	SELECT
		 ст.Наименование AS Территория
		, сг.Ссылка AS СсылкаГорода
	FROM dwh.[Справочник.Города] сг
	JOIN dwh.[Справочник.Фирмы] сф
		ON сф.Город = сг.Ссылка
	JOIN dwh.[Справочник.Фирмы.ИерархияТерриторий] сфит
		ON сф.Ссылка = сфит.Ссылка
		AND сфит.Уровень = 3
	JOIN dwh.[Справочник.Территории] ст
		ON ст.Ссылка = сфит.Территория
	GROUP BY
		 ст.Наименование
		, сг.Ссылка 	
		)	
SELECT
	  Территория
	, Город
	, КодГорода
	, Должность
	, findMaxDate.Уровень
	, Зарплата
	, CONVERT(DATE, GETDATE()) AS ДатаОбновления
	, ЛидерскийУровень
FROM findMaxDate
JOIN joinTerrerory
	ON СсылкаГород = joinTerrerory.СсылкаГорода

			 """
    df = execute_onec(query)
    return df

In [84]:
def getActualStaff():
    query = f"""
select
     as2."Должность"
    , as2."УровеньОплаты"
    , as2."ТекущийФилиал"
    ,count(*) as КоличествоСотрудников
    , as2."Город"
    , as2."Филиал"
  from actual_staff as2
  where as2."Должность" not like '%%(на аутсорсе)%%'
  group by as2."Должность", as2."УровеньОплаты", as2."ТекущийФилиал", as2."ДатаОбновления", as2."Город", as2."Филиал", as2."РРС" 
  order by as2."Должность", as2."ТекущийФилиал", as2."УровеньОплаты" 
  """
    df = execute_pg_hr(query)
    return df

In [85]:
def getLossBranches():

	"""НазваниеФирмы	НазваниеГорода	Территория	Должность	Уровень"""

	query = f"""
	
WITH МаксДатаЦТЕ AS (
	select 
	--	DATEFROMPARTS(YEAR(Дата), MONTH(Дата), 1) AS Дата
			Дата
		, сф.Наименование AS НазваниеФирмы 
		, сг.Код AS КодГорода 
		, сг.Наименование as НазваниеГорода
		, сф.Код as Фирма
		, сшр.Наименование as Должность
		, душрс.КоличествоСтавок
		, max(Дата) OVER (PARTITION BY сф.Код, сшр.Наименование) AS МаксДата
	from dwh.[Документ.УтверждениеШтатногоРасписания] душр
	join (
		select Фирма, YEAR(Дата) as Год, MONTH(Дата) as Месяц, MAX(DAY(Дата)) as День
		from dwh.[Документ.УтверждениеШтатногоРасписания] душрa 
		group by Фирма, YEAR(Дата), MONTH(Дата)
		) tmp
		on tmp.Год = YEAR(душр.Дата)
		and tmp.Месяц = MONTH(душр.Дата)
		and tmp.День = DAY(душр.Дата)
		and tmp.Фирма = душр.Фирма 
	join dwh.[Документ.УтверждениеШтатногоРасписания.Состав] душрс
		on душрс.Ссылка = душр.Ссылка
	join dwh.[Справочник.Фирмы] сф 
		on сф.Ссылка = душр.Фирма 
	join dwh.[Справочник.ШтатноеРасписание] сшр 
		on сшр.Ссылка = душрс.Должность 
	JOIN dwh.[Справочник.Города] сг 
		ON сф.Город = сг.Ссылка 
	), result_table AS
	(
	SELECT НазваниеФирмы, НазваниеГорода, Должность
	FROM МаксДатаЦТЕ
	WHERE МаксДата = Дата
	group by НазваниеФирмы, НазваниеГорода, Должность
	)

	select rt1.*
		, 1 as Уровень
	from result_table rt1
	union all 
	select rt2.*
		, 2 as Уровень
	from result_table rt2
	union all
	select rt3.*
		, 3 as Уровень
	from result_table rt3			

	"""
	df = execute_onec(query)
	return df

In [86]:
df_PosLvlSalary = GetPosLvlSalary()
df_PosLvlSalary

,Территория,Город,КодГорода,Должность,Уровень,ДатаОбновления,ЛидерскийУровень
0,РРС ПР Нижегородская область,Семенов,509.0,Старший кладовщик магазина,1.0,2022-07-22,2.0
1,РРС ПР Нижегородская область,Семенов,509.0,Старший кладовщик магазина,2.0,2022-07-22,2.0
2,РРС ПР Нижегородская область,Семенов,509.0,Старший кладовщик магазина,3.0,2022-07-22,2.0
3,РРС ПР Нижегородская область,Семенов,509.0,Кредитный специалист,1.0,2022-07-22,1.0
4,РРС ПР Нижегородская область,Семенов,509.0,Кредитный специалист,2.0,2022-07-22,1.0
...,...,...,...,...,...,...,...
173087,РРС ЗС Томск,Тисуль,1913.0,Универсальный сотрудник магазина,2.0,2022-07-22,1.0
173088,РРС ЗС Томск,Тисуль,1913.0,Универсальный сотрудник магазина,3.0,2022-07-22,1.0
173089,РРС ЗС Томск,Тисуль,1913.0,Управляющий маг. средней категории,1.0,2022-07-22,2.0
173090,РРС ЗС Томск,Тисуль,1913.0,Управляющий маг. средней категории,2.0,2022-07-22,2.0


In [87]:
df_actual_staff = getActualStaff()
df_actual_staff

,Должность,УровеньОплаты,ТекущийФилиал,КоличествоСотрудников,Город,Филиал
0,Администратор киностудии,2.0,2421.0,1,Владивосток,Видеостудия Владивосток
1,Администратор офиса,2.0,284.0,1,Москва,Адм. дивизиона Центральный
2,Администратор офиса,3.0,292.0,1,Воронеж,Адм. дивизиона Черноземье
3,Администратор офиса,1.0,897.0,1,Новосибирск,Адм. дивизиона Западная Сибирь
4,Администратор офиса,2.0,897.0,1,Новосибирск,Адм. дивизиона Западная Сибирь
...,...,...,...,...,...,...
18553,Юрист,3.0,5822.0,1,Владивосток,Направление ЮО по работе с поставщиками
18554,Юрист,1.0,5823.0,2,Владивосток,Направление ЮО по работе с розницей
18555,Юрист,2.0,5823.0,1,Владивосток,Направление ЮО по работе с розницей
18556,Юрист,3.0,5823.0,1,Владивосток,Направление ЮО по работе с розницей


In [88]:
df_loss_branches = getLossBranches()
df_loss_branches

,НазваниеФирмы,НазваниеГорода,Должность,Уровень
0,Федеральная дирекция,РКЦ Владивосток,Директор,1
1,Федеральная дирекция,РКЦ Владивосток,Директор по закупкам,1
2,Федеральная дирекция,РКЦ Владивосток,Руководитель службы обучения и развития персонала,1
3,Федеральная дирекция,РКЦ Владивосток,Федеральный бизнес аналитик,1
4,Федеральная дирекция,РКЦ Владивосток,Федеральный директор,1
...,...,...,...,...
60919,Яшкино в Центре,Яшкино,Зам. управляющего магазина,3
60920,Яшкино в Центре,Яшкино,Универсальный сотрудник магазина,3
60921,Яшкино в Центре,Яшкино,Универсальный сотрудник магазина 1 категории,3
60922,Яшкино в Центре,Яшкино,Универсальный сотрудник магазина 2 категории,3


In [89]:
df_main =  df_loss_branches\
    .merge(df_actual_staff, left_on=['НазваниеФирмы', 'Должность', 'Уровень'], right_on=['Филиал', 'Должность', 'УровеньОплаты'], how='left')
df_main['Город'] = df_main['Город'].fillna(df_main['НазваниеГорода'])
df_main['Филиал'] = df_main['Филиал'].fillna(df_main['НазваниеФирмы'])
df_main['КоличествоСотрудников'] = df_main['КоличествоСотрудников'].fillna(0)
df_main

,НазваниеФирмы,НазваниеГорода,Должность,Уровень,УровеньОплаты,ТекущийФилиал,КоличествоСотрудников,Город,Филиал
0,Федеральная дирекция,РКЦ Владивосток,Директор,1,NaN,NaN,0.0,РКЦ Владивосток,Федеральная дирекция
1,Федеральная дирекция,РКЦ Владивосток,Директор по закупкам,1,NaN,NaN,0.0,РКЦ Владивосток,Федеральная дирекция
2,Федеральная дирекция,РКЦ Владивосток,Руководитель службы обучения и развития персонала,1,NaN,NaN,0.0,РКЦ Владивосток,Федеральная дирекция
3,Федеральная дирекция,РКЦ Владивосток,Федеральный бизнес аналитик,1,NaN,NaN,0.0,РКЦ Владивосток,Федеральная дирекция
4,Федеральная дирекция,РКЦ Владивосток,Федеральный директор,1,NaN,NaN,0.0,РКЦ Владивосток,Федеральная дирекция
...,...,...,...,...,...,...,...,...,...
61079,Яшкино в Центре,Яшкино,Зам. управляющего магазина,3,NaN,NaN,0.0,Яшкино,Яшкино в Центре
61080,Яшкино в Центре,Яшкино,Универсальный сотрудник магазина,3,NaN,NaN,0.0,Яшкино,Яшкино в Центре
61081,Яшкино в Центре,Яшкино,Универсальный сотрудник магазина 1 категории,3,3.0,3784.0,1.0,Яшкино,Яшкино в Центре
61082,Яшкино в Центре,Яшкино,Универсальный сотрудник магазина 2 категории,3,NaN,NaN,0.0,Яшкино,Яшкино в Центре


In [90]:
df_checkerboard_salary = df_PosLvlSalary.merge(df_main, left_on=['Город','Должность', 'Уровень'], right_on=['Город', 'Должность', 'Уровень'], how='left')
df_checkerboard_salary = df_checkerboard_salary[['Территория','Город', 'Филиал', 'Должность', 
                        'Уровень', 'Зарплата','ЛидерскийУровень','КоличествоСотрудников']]

In [91]:
df_checkerboard_salary[df_checkerboard_salary['Филиал'] == 'Владивосток Галактика КБТ'].head(270)

,Территория,Город,Филиал,Должность,Уровень,ЛидерскийУровень,КоличествоСотрудников
62274,див. ДВ (B2B),Владивосток,Владивосток Галактика КБТ,Консультант дискаунтера,1.0,1.0,0.0
62277,див. ДВ (DNS Дом),Владивосток,Владивосток Галактика КБТ,Консультант дискаунтера,1.0,1.0,0.0
62280,див. ДВ (Логистика),Владивосток,Владивосток Галактика КБТ,Консультант дискаунтера,1.0,1.0,0.0
62283,див. ДВ (Сервис),Владивосток,Владивосток Галактика КБТ,Консультант дискаунтера,1.0,1.0,0.0
62286,див. ДВ (Управление),Владивосток,Владивосток Галактика КБТ,Консультант дискаунтера,1.0,1.0,0.0
...,...,...,...,...,...,...,...
84889,Техноструктура,Владивосток,Владивосток Галактика КБТ,Управляющий маг. средней категории,3.0,2.0,0.0
84907,Фед. контактный центр,Владивосток,Владивосток Галактика КБТ,Управляющий маг. средней категории,3.0,2.0,0.0
84925,Федеральный распределительный центр,Владивосток,Владивосток Галактика КБТ,Управляющий маг. средней категории,3.0,2.0,0.0
84943,"РРС ДВ ""Владивосток""",Владивосток,Владивосток Галактика КБТ,Управляющий маг. средней категории,3.0,2.0,0.0


In [92]:
df_checkerboard_salary

,Территория,Город,Филиал,Должность,Уровень,ЛидерскийУровень,КоличествоСотрудников
0,РРС ПР Нижегородская область,Семенов,Семенов ТЦ Плаза,Старший кладовщик магазина,1.0,2.0,0.0
1,РРС ПР Нижегородская область,Семенов,Семенов ТЦ Плаза,Старший кладовщик магазина,2.0,2.0,1.0
2,РРС ПР Нижегородская область,Семенов,Семенов ТЦ Плаза,Старший кладовщик магазина,3.0,2.0,0.0
3,РРС ПР Нижегородская область,Семенов,NaN,Кредитный специалист,1.0,1.0,NaN
4,РРС ПР Нижегородская область,Семенов,NaN,Кредитный специалист,2.0,1.0,NaN
...,...,...,...,...,...,...,...
383392,РРС ЗС Томск,Тисуль,Тисуль в Центре,Универсальный сотрудник магазина,2.0,1.0,0.0
383393,РРС ЗС Томск,Тисуль,Тисуль в Центре,Универсальный сотрудник магазина,3.0,1.0,0.0
383394,РРС ЗС Томск,Тисуль,Тисуль в Центре,Управляющий маг. средней категории,1.0,2.0,0.0
383395,РРС ЗС Томск,Тисуль,Тисуль в Центре,Управляющий маг. средней категории,2.0,2.0,1.0


In [93]:
df_checkerboard_salary.isna().sum() # 3898

Территория                   0
Город                        0
Филиал                   74604
Должность                    0
Уровень                      0
ЛидерскийУровень             0
КоличествоСотрудников    74604
dtype: int64

In [94]:
filt_test1 = df_test_1['УровеньОплаты'].isna()
df_test_1[filt_test1]

NameError: name 'df_test_1' is not defined

In [ ]:
# filt = (df_checkerboard_salary['НазваниеФирмы'] == 'Владивосток Галактика КБТ') & (df_checkerboard_salary['Филиал'] == 'Владивосток Галактика КБТ') 
filt1 = (df_test_1['НазваниеФирмы'] == 'Владивосток Галактика КБТ')
df_test_1[filt1].head(135).sort_values(by=['Должность', 'Уровень'])



NameError: name 'df_test_1' is not defined

In [ ]:
df_checkerboard_salary['Филиал'].isna().sum()

35682

In [ ]:
df_checkerboard_salary['Филиал'].isna().sum()

35682

In [ ]:
df_checkerboard_salary['Филиал'] = df_checkerboard_salary['Филиал'].fillna(df_checkerboard_salary['НазваниеФирмы'])

In [ ]:
df = df_checkerboard_salary[['Город','Должность', 'Уровень', 'Зарплата', 'Филиал', 'ЛидерскийУровень', 'КоличествоСотрудников']]

In [ ]:
qqq = df_checkerboard_salary.groupby(['Город','Должность', 'Уровень', 'Зарплата', 'Филиал', 'ЛидерскийУровень', 'КоличествоСотрудников'], as_index=False)['Город','Должность', 'Уровень', 'Зарплата', 'Филиал', 'ЛидерскийУровень', 'КоличествоСотрудников']

C:\Users\peredereev.ro\AppData\Local\Temp\ipykernel_14816\461515408.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  qqq = df_checkerboard_salary.groupby(['Город','Должность', 'Уровень', 'Зарплата', 'Филиал', 'ЛидерскийУровень', 'КоличествоСотрудников'], as_index=False)['Город','Должность', 'Уровень', 'Зарплата', 'Филиал', 'ЛидерскийУровень', 'КоличествоСотрудников']


In [ ]:
qqq.size()

,Город,Должность,Уровень,Зарплата,Филиал,ЛидерскийУровень,КоличествоСотрудников,size
0,Абаза,Универсальный сотрудник магазина,1.0,31000.0,Абаза ТК Околица,1.0,1.0,1
1,Абаза,Универсальный сотрудник магазина,2.0,34000.0,Абаза ТК Околица,1.0,3.0,1
2,Абаза,Универсальный сотрудник магазина,3.0,37000.0,Абаза ТК Околица,1.0,0.0,1
3,Абаза,Управляющий маг.,1.0,55000.0,Абаза ТК Околица,2.0,0.0,1
4,Абаза,Управляющий маг.,2.0,60000.0,Абаза ТК Околица,2.0,1.0,1
...,...,...,...,...,...,...,...,...
41602,Яшкино,Универсальный сотрудник магазина 2 категории,2.0,25400.0,Яшкино в Центре,1.0,1.0,1
41603,Яшкино,Универсальный сотрудник магазина 2 категории,3.0,28000.0,Яшкино в Центре,1.0,0.0,1
41604,Яшкино,Управляющий маг. средней категории,1.0,65000.0,Яшкино в Центре,2.0,0.0,1
41605,Яшкино,Управляющий маг. средней категории,2.0,80000.0,Яшкино в Центре,2.0,1.0,1


In [ ]:
df_checkerboard_salary[df_checkerboard_salary['Зарплата'].isna()]['Город'].unique()


array([], dtype=object)

In [ ]:
# filt = df_checkerboard_salary['Город'] == 'Владивосток' 
# df_checkerboard_salary.loc[filt][df_checkerboard_salary['Зарплата'].isna()]
# qqq = qqq[qqq['Филиал'].isna()]
# qqq